In [2]:
import numpy as np

In [3]:
X_train, y_train, X_test, y_test, X_dev, y_dev = np.load('./all_data.npy', allow_pickle = True)

In [4]:
def make_X(x):
    res = []
    x = x[767:]
    for i in range(0, x.shape[0], 768):
#         print(i)
        res.append(np.array(x[i:i+768]))
    print(np.array(res).shape)
    return np.array(res)

In [5]:
X_train, X_test, X_dev = make_X(X_train), make_X(X_test), make_X(X_dev)

(9989, 768)
(2610, 768)
(1109, 768)


In [6]:
from sklearn import preprocessing

min_max_scaler = preprocessing.StandardScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)
X_dev = min_max_scaler.transform(X_dev)

In [7]:
num_classes = 7
time_step = 30

In [8]:
def expand_on_time(X, timestep = time_step):
    tmp = np.reshape(X[0], (1, len(X[0])))
#     print(tmp.shape)
    tmp = np.repeat(tmp, timestep, axis = 0)
#     print(tmp.shape)
    ab = np.reshape(tmp, (1, len(tmp[:,:]), len(tmp[0])))
#     print(ab.shape)
#     print(X.shape)
    X = np.repeat(ab, X.shape[0], axis = 0)
#     print(X.shape)
    return X

In [9]:
# X_train, X_test, X_dev = expand_on_time(X_train), expand_on_time(X_test), expand_on_time(X_dev)

In [10]:
import pandas as pd

loly = pd.DataFrame(y_train, columns = range(0,7))

In [11]:
arr = [4.0, 15.0, 15.0, 3.0, 1.0, 6.0, 3.0]

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Bidirectional, Input
from tensorflow.keras.optimizers import RMSprop, SGD

In [13]:
X_train.shape

(9989, 768)

In [14]:
import tensorflow as tf

with tf.device('/GPU:0'):

    model = Sequential()
#     model.add(LSTM(800, input_shape=(time_step, 768), return_sequences=True))
#     model.add(LSTM(10))
#     model.add(Dropout(0.5))
    # model.add(LSTM(64, return_sequences = True))
#     model.add(Dropout(0.4))
#     model.add(LSTM(150, return_sequences = True))
#     model.add(LSTM(32))
#     model.add(Dropout(0.5))
    # model.add(Activation('softmax'))
    # model.add(Dense(32, activation = 'tanh'))
    # model.add(Dropout(0.4))
    model.add(Input(shape = (768,)))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(num_classes, activation = 'softmax'))

    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    #rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               393728    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 4

In [16]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10)


In [17]:
class_weights = {}

for i in range(7):
    class_weights[i] = arr[i]

In [18]:
class_weights

{0: 4.0, 1: 15.0, 2: 15.0, 3: 3.0, 4: 1.0, 5: 6.0, 6: 3.0}

In [25]:
X_train = np.random.rand(X_train.shape[0], X_train.shape[1])

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
# plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def get_tfarray(arr):
    result = []
    for i in arr:
        m = np.max(i)
        result.append(i == m)
    return np.array(result)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
class_names = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']


y_pred = model.predict(X_test)

y_pred = get_tfarray(y_pred)

accuracy_score(y_test, y_pred, normalize=False)
print(classification_report(y_test, y_pred, target_names=class_names, digits=4))

In [ ]:
from sklearn.metrics import confusion_matrix
#Get the confusion matrix


#array([[1, 0, 0],
#   [1, 0, 0],
#   [0, 1, 2]])
cm =confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
#Now the normalize the diagonal entries
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#array([[1.        , 0.        , 0.        ],
#      [1.        , 0.        , 0.        ],
#      [0.        , 0.33333333, 0.66666667]])

#The diagonal entries are the accuracies of each class
cm.diagonal()

In [ ]:
predictions = model.predict(X_test)

In [ ]:
y_test[0], predictions[0]

In [ ]:
y_test[-1], predictions[-1]

In [ ]:
np.argmax(y_test, axis = 1), np.argmax(predictions, axis = 1)